**TRIPS & WEATHER DATASET**

In [ ]:
# Import necessary packages
import polars as pl

# Load the bike trips and weather data
trips = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_trips.csv", null_values='NA')
weather = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_weather.csv", null_values=['NA',''])

trips = trips.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
weather = weather.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
print(trips.height)
print(weather.height)

78704
733


In [ ]:
#Question 1, Part A
#The trips data represent every ride taken for a bike share company.
#The weather data represent the prevailing weather for a particular day.
#Join the trips and weather data together using the date column. Name the resulting DataFrame trips_weather.

#Insert a code block below that joins trips to weather using the date column.

# Join trips and weather on the 'date' column
trips_weather = trips.join(weather, on="date", how="inner")

print(trips_weather.head())
print(trips_weather.height)

shape: (5, 35)
┌──────┬──────────┬─────────────┬─────────────┬───┬─────────────┬────────┬────────────┬────────────┐
│ id   ┆ duration ┆ start_date  ┆ start_stati ┆ … ┆ cloud_cover ┆ events ┆ wind_dir_d ┆ zip_code_r │
│ ---  ┆ ---      ┆ ---         ┆ on_name     ┆   ┆ ---         ┆ ---    ┆ egrees     ┆ ight       │
│ i64  ┆ i64      ┆ str         ┆ ---         ┆   ┆ i64         ┆ str    ┆ ---        ┆ ---        │
│      ┆          ┆             ┆ str         ┆   ┆             ┆        ┆ i64        ┆ i64        │
╞══════╪══════════╪═════════════╪═════════════╪═══╪═════════════╪════════╪════════════╪════════════╡
│ 4721 ┆ 3        ┆ 2013-08-29T ┆ Market at   ┆ … ┆ 4           ┆ null   ┆ 286        ┆ 94107      │
│      ┆          ┆ 20:27:00Z   ┆ 10th        ┆   ┆             ┆        ┆            ┆            │
│ 4812 ┆ 3        ┆ 2013-08-29T ┆ 2nd at      ┆ … ┆ 4           ┆ null   ┆ 286        ┆ 94107      │
│      ┆          ┆ 21:30:00Z   ┆ Folsom      ┆   ┆             ┆        ┆  

In [ ]:
#Question 1, Part B

# Filter for days with a weather event (non-null 'events') and compute average duration
avg_bad_weather = (
    trips_weather
    .filter(pl.col("events").is_not_null())
    .select(pl.col("duration").mean().alias("avg_duration"))
)

print(avg_bad_weather)

shape: (1, 1)
┌──────────────┐
│ avg_duration │
│ ---          │
│ f64          │
╞══════════════╡
│ 10.34899     │
└──────────────┘


**BASEBALL DATASET**

In [ ]:
# Load baseball datasets
Batting = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv", null_values='NA')
People = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/People.csv", null_values='NA')
HallOfFame = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv", null_values='NA')

In [ ]:
# Question 2, Part A
#Join Batting with People on playerID
stats_w_bio = Batting.join(
    People,
    left_on="playerID",
    right_on="playerID",
    how="left"  # Use a left join to keep all batting stats
)

# Display the first few rows to verify
stats_w_bio.head()

playerID,yearID,stint,teamID,lgID,G,AB,R,H,X2B,X3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,deathDate,birthDate
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str
"""abercda01""",1871,1,"""TRO""",null,1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0,1850,1,2,"""USA""","""OK""","""Fort Towson""",1939,11,11,"""USA""","""PA""","""Philadelphia""","""Frank""","""Abercrombie""","""Francis Patterson""",null,null,null,null,"""1871-10-21""","""1871-10-21""","""aberd101""","""abercda01""","""1939-11-11""","""1850-01-02"""
"""addybo01""",1871,1,"""RC1""",null,25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0,1842,2,null,"""CAN""","""ON""","""Port Hope""",1910,4,9,"""USA""","""ID""","""Pocatello""","""Bob""","""Addy""","""Robert Edward""",160,68,"""L""","""L""","""1871-05-06""","""1877-10-06""","""addyb101""","""addybo01""","""1910-04-09""",null
"""allisar01""",1871,1,"""CL1""",null,29,137,28,40,4,5,0,19,3,1,2,5,null,null,null,null,1,1849,1,29,"""USA""","""PA""","""Philadelphia""",1916,2,25,"""USA""","""DC""","""Washington""","""Art""","""Allison""","""Arthur Algernon""",150,68,null,null,"""1871-05-04""","""1876-10-05""","""allia101""","""allisar01""","""1916-02-25""","""1849-01-29"""
"""allisdo01""",1871,1,"""WS3""",null,27,133,28,44,10,2,2,27,1,1,0,2,null,null,null,null,0,1846,7,12,"""USA""","""PA""","""Philadelphia""",1916,12,19,"""USA""","""DC""","""Washington""","""Doug""","""Allison""","""Douglas L.""",160,70,"""R""","""R""","""1871-05-05""","""1883-07-13""","""allid101""","""allisdo01""","""1916-12-19""","""1846-07-12"""
"""ansonca01""",1871,1,"""RC1""",null,25,120,29,39,11,3,0,16,6,2,2,1,null,null,null,null,0,1852,4,17,"""USA""","""IA""","""Marshalltown""",1922,4,14,"""USA""","""IL""","""Chicago""","""Cap""","""Anson""","""Adrian Constantine""",227,72,"""R""","""R""","""1871-05-06""","""1897-10-03""","""ansoc101""","""ansonca01""","""1922-04-14""","""1852-04-17"""


In [ ]:
# Filter for players born in Florida and sum their HR
florida_hr_total = (
    stats_w_bio
    .filter(pl.col("birthState") == "FL")  # Filter players born in Florida
    .select(pl.col("HR"))                  # Select the HR column
    .sum()                                 # Sum the HRs
)

print(florida_hr_total)

shape: (1, 1)
┌───────┐
│ HR    │
│ ---   │
│ i64   │
╞═══════╡
│ 16225 │
└───────┘


In [ ]:
# Question 2, part B

# Calculate total home runs for players born in Florida
florida_hr = (
    stats_w_bio
    .filter(pl.col("birthState") == "FL")   # Filter for Florida-born players
    .select(pl.col("HR").sum().alias("total_hr"))  # Sum HR and rename column
)

florida_hr

total_hr
i64
16225


In [ ]:
# Question 3, part A

# Filter HallOfFame for inducted players in the Player category
hall_inducted = (
    HallOfFame
    .filter((pl.col("inducted") == "Y") & (pl.col("category") == "Player"))
    .select("playerID")   # Keep only the playerID column
    .unique()             # Ensure each playerID is unique
)

hall_inducted

playerID
str
"""ryanno01"""
"""brettge01"""
"""stargwi01"""
"""molitpa01"""
"""drysddo01"""
…
"""boudrlo01"""
"""vaughar01"""
"""gwynnto01"""


In [ ]:
# Join Batting with hall_inducted to get stats only for Hall of Famers
hof_batting_stats = Batting.join(
    hall_inducted,
    on="playerID",
    how="inner"  # Keep only players who are in hall_inducted
)

# Calculate total career hits per player
career_hits = (
    hof_batting_stats
    .group_by("playerID")
    .agg(pl.col("H").sum().alias("career_hits"))
)

# Calculate the average career hits for all Hall of Famers
average_career_hits = career_hits.select(pl.col("career_hits").mean().alias("avg_career_hits"))

average_career_hits

avg_career_hits
f64
1717.29918


In [ ]:
# Question 3, part B

# Join Batting with hall_inducted to get only Hall of Fame players' stats
hof_batting_stats1 = Batting.join(
    hall_inducted,
    on="playerID",
    how="inner"
)

# Calculate total career hits per player
career_hits = hof_batting_stats.group_by("playerID").agg(
    pl.col("H").sum().alias("total_hits")
)

# Calculate the average career hits across all Hall of Famers
hof_hits = career_hits.select(
    pl.col("total_hits").mean().alias("average_hits")
)

hof_hits

average_hits
f64
1717.29918
